# 潜在変数モデルと混合モデル

潜在変数モデルと混合モデルでは、生成の仕組みを抽象用語で終わらせず、毎段階をコードで確認します。


## 手を動かす 1: 混合分布を意識したサンプル

潜在変数と混合モデルの違いを掴むため、2モードの潜在サンプルを作ります。


In [ ]:
import random
random.seed(13)
z = [round(random.choice([-1.5, 1.8]) + random.gauss(0, 0.15), 3) for _ in range(5)]
print('task = latent-variable-mixture-models')
print('latent z =', z)

モード構造が生成結果へどう影響するかを見ます。

この節では、潜在変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 手を動かす 2: 単純なデコーダを書く

次に、潜在変数を観測空間へ写像する簡易デコーダを作ります。生成モデルの基本構造をコードで可視化します。


In [ ]:
weights = [1.4, -0.6, 0.8, 0.5, -1.1]
bias = 0.2
x_hat = sum(zi * wi for zi, wi in zip(z, weights)) + bias
print('decoded scalar =', round(x_hat, 5))
print('abs scale =', round(abs(x_hat), 5))

実際の生成モデルは高次元ですが、構造は同じです。潜在を観測へ写像し、再構成品質を改善します。

この節では、潜在変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 数式メモ

1. $p_{\theta}(x) = \int p_{\theta}(x\mid z)\,p(z)\,dz$
2. $s(x) = \nabla_x \log p(x)$


## 手を動かす 3: ノイズを加えて復元する

ここで、拡散系モデルの直感を最小実験で確認します。ノイズ付加と復元の往復を短いコードで体験します。


In [ ]:
x0 = 1.5
beta = 0.12
noise = -0.3
xt = ((1 - beta) ** 0.5) * x0 + (beta ** 0.5) * noise
x0_hat = (xt - (beta ** 0.5) * noise) / ((1 - beta) ** 0.5)
print('x0, xt, x0_hat =', round(x0, 5), round(xt, 5), round(x0_hat, 5))

復元誤差を観察すると、ノイズスケジュールの意味が見えてきます。理論と実装をつなぐ重要な観測点です。

この節では、潜在変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 手を動かす 4: 混合分布の感覚を作る

次に、複数モードを持つ分布を手で作ります。モード崩壊の議論に入る前の下地として有効です。


In [ ]:
mix = [(-2.0, 0.4), (1.5, 0.6)]
samples = []
for m, w in mix:
    samples.append(round(m + (w * 0.1), 3))
print('mode-aware samples =', samples)

混合分布の直感があると、生成結果の『多様性』を定量評価する発想が自然になります。

この節では、潜在変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 手を動かす 5: 学習指標を定義する

最後に、生成品質を観察する最小指標を作ります。見た目だけで判断しない習慣を作ることが狙いです。


In [ ]:
recon_errors = [0.42, 0.31, 0.29, 0.36, 0.33]
avg = sum(recon_errors) / len(recon_errors)
worst = max(recon_errors)
best = min(recon_errors)
print('avg/best/worst =', round(avg, 4), round(best, 4), round(worst, 4))

平均値と外れ値を同時に見ると、モデルが安定しているかを判断しやすくなります。

この節では、潜在変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## この節の要点

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 見た目の良さだけで比較してしまう
2. 多様性と品質のトレードオフを観測しない
3. ノイズスケジュールの意味を理解しないまま調整する
